## Chat Completion agent

[MS Learn sample](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/chat-completion-agent?pivots=programming-language-python)

In [5]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from datetime import datetime

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

load_dotenv()

True

In [16]:
chatServiceName = "chatService"
writerName = "writer"
reviewerName = "reviewer"
writerInstructions = "You are an gent designed to express your thoughts and feelings in a way that is clear and concise. You are a writer."
reviewerInstructions = """You are an agent designed to provide feedback on the clarity and conciseness of the thoughts and feelings expressed by the writer.
    Respond with critique and instructions to the writer on how to improve their message.
    When you consider the writer's message to be clear and concise, you should respond with "Approved."
    You are a reviewer."""

In [13]:
chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    service_id=chatServiceName, # Optional; for targeting specific services within Semantic Kernel
)

In [20]:
# Define the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
kernel.add_service(AzureChatCompletion(service_id="chatService"))

# Create the agent
writer = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=writerName, 
  instructions=writerInstructions,
)
reviewer = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=reviewerName, 
  instructions=reviewerInstructions,
)

In [21]:
history = ChatHistory()
user_input = input("User:> ")
if user_input:
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
    is_complete: bool = False
    while not is_complete:
        async for response in writer.invoke(history=history):
            print(f"Writer: {response.content}")
            history.add_message(ChatMessageContent(role=AuthorRole.ASSISTANT, content=response.content))
            last_proposal = response.content
        async for response in reviewer.invoke(history=history):
            print(f"Reviewer: {response.content}") 
            history.add_message(ChatMessageContent(role=AuthorRole.ASSISTANT, content=response.content))                       
        if response.content == "Approved.":
            is_complete = True
            break
print("Approved version:", last_proposal)


Writer: The sailboat floated silently on the calm water, its white sails drooping like tired wings. The sun dipped low on the horizon, casting a warm golden hue across the sky, blending with shades of orange and pink. The world felt serene, suspended in a moment where time seemed to stand still. 

Without the momentum of wind, the tranquil surface of the water mirrored the vibrant colors above, creating a breathtaking canvas. Soft whispers of the evening air rustled through the rigging, as if nature itself sighed in contentment. 

In that stillness, the sailboat became a symbol of peaceful solitude, a reminder of the beauty in quiet moments and the magic of sunsets that linger long after the day has ended.
Reviewer: Your writing beautifully captures the imagery and emotion of a sailboat on a windless day at sunset. However, to enhance clarity and conciseness, consider the following suggestions:

1. **Focus on Core Imagery:** While your descriptions are vivid, some phrases could be stre